In [40]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pathlib import Path
import requests
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
print("Setup Complete")

Setup Complete


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
RAW_URL = "https://raw.githubusercontent.com/brick1ng5654/course-3/RafaelB/boml/lab_01/health_lifestyle_classification.csv"
DEST = Path("/content/data/health_lifestyle_classification.csv")
DEST.parent.mkdir(parents=True, exist_ok=True)

r = requests.get(RAW_URL, timeout=60)
r.raise_for_status()

DEST.write_bytes(r.content)
print(f"Saved to {DEST} ({DEST.stat().st_size/1_000_000:.2f} MB)")

Saved to /content/data/health_lifestyle_classification.csv (52.98 MB)


In [35]:
# Path of the file to read
filepath = "/content/data/health_lifestyle_classification.csv"
# Read the file into a variable DF
DF = pd.read_csv(filepath)
DF.isna().sum()


,0
survey_code,0
age,0
gender,0
height,0
weight,0
bmi,0
bmi_estimated,0
bmi_scaled,0
bmi_corrected,0
waist_size,0


In [37]:
delete_columns = [
    'screen_time',
    'family_history',
    'mental_health_score',
    'occupation',
    'mental_health_support',
    'device_usage',
    'healthcare_access',
    'insurance',
    'pet_owner',
    'height',
    'weight',
    'waist_size',
    'bmi',
    'physical_activity',
    'education_level',
    'job_type',
    'income',
    'electrolyte_level',
    'gene_marker_flag',
    'environmental_risk_score',
    'daily_supplement_dosage'
]

DF_cp = DF.copy()

for column in delete_columns:
    DF_cp = DF_cp.drop(column, axis=1)

for column in DF_cp.columns:
  if DF_cp[column].dtype in ["int64","float64"]:
    medidan = DF_cp[column].median()
    DF_cp[column] = DF_cp[column].fillna(medidan)
  else:
    DF_cp.dropna(subset=[column],inplace=True)

In [38]:
def coding_categorial(series):
    frequency_mapping = {
        "Male": 0, "Female": 1,
        "Poor": 0, "Fair": 1, "Good": 2, "Excellent": 3,
        "Occasionally": 0, "Regularly": 1,
        "Non-smoker": 0, "Light": 1, "Heavy": 2,
        "No": 0, "Yes": 1,
        "High School": 0, "Bachelor": 1, "Master": 2, "PhD": 3,
        "Tech": 0, "Office": 1, "Labor": 2, "Unemployed": 3, "Service": 4, "Healthcare": 5,
        "Vegan": 0, "Vegetarian": 1, "Omnivore": 2, "Keto": 3,
        "Strength": 0, "Cardio": 1, "Mixed": 2,
        "Low": 0, "Moderate": 1, "High": 2,
        "diseased": 0, "healthy": 1
    }
    return series.map(frequency_mapping)

    for column in DF_cp.columns:
      if DF_cp[column].dtype not in ["int64","float64"]:
        DF_cp[column] = coding_categorial(DF_cp[column])


In [44]:
categorical_col = DF_cp.select_dtypes(include=["object"]).columns

for column in DF_cp.columns:
  if column not in categorical_col:
    min_max_scaler = MinMaxScaler()
    DF_cp[column] = min_max_scaler.fit_transform(DF_cp[[column]])


In [47]:
DF_end = DF_cp.drop_duplicates()
